In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, IterableDataset
from tqdm import tqdm
import numpy as np
import json
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import os
import math
from collections import defaultdict
import pandas as pd

%matplotlib inline

2025-11-22 17:21:12.001358: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seeds for reproducibility
seed = 99
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [3]:
class IterTensorFlowToPyTorchDataset(IterableDataset):
    def __init__(self, tf_dataset_path):
        """
        A PyTorch dataset that fetches data from a TensorFlow dataset.
        
        Args:
        - tf_dataset: A TensorFlow dataset.
        """
        self.tf_path = tf_dataset_path
        self.tf_dataset = tf.data.Dataset.load(tf_dataset_path)
        
        
    def __len__(self):
        return tf.data.experimental.cardinality(self.tf_dataset).numpy()
            
    def __iter__(self):
        for element in self.tf_dataset.as_numpy_iterator():
            features, labels = element
            features = torch.tensor(features, dtype=torch.float32).view(1, -1)  # Reshape to match model input shape
            labels = torch.tensor(labels, dtype=torch.float32)
            
            # # Normalize the labels
            # labels = self.output_normalization(labels)
    
            yield features, labels
    
    def size(self, in_gb=False):
        ''' Returns the size of the Dataset in MB or GB.
        '''
        if os.path.exists(self.tf_path):
            if os.path.isfile(self.tf_path):
                # If it's a single file
                size_in_bytes = os.path.getsize(self.tf_path)
            elif os.path.isdir(self.tf_path):
                # If it's a directory, sum up the sizes of all files inside it
                size_in_bytes = sum(
                    os.path.getsize(os.path.join(root, file))
                    for root, _, files in os.walk(self.tf_path)
                    for file in files
                )
            else:
                raise ValueError(f"Path '{self.tf_path}' is neither a file nor a directory.")
            
                            
            if in_gb == False:
                size_in_mb = size_in_bytes / (1024*1024) # Convert to MB
                return size_in_mb
            else:
                size_in_gb = size_in_bytes / (1024*1024*1024) # Convert to GB
                return size_in_gb
        else:
            raise FileNotFoundError(f"Data file not found at {self.tf_path}")


In [4]:
# Deep model
class DeepModel(nn.Module):
    """
    Deep CNN model for regression.
    
    The model requires initialization before loading weights. 
    To initialize the model, call model.initialize(input_tensor).
    """
    def __init__(self, num_vars):
        super(DeepModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=16, stride=1, dilation=1)
        # self.bn1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=16, stride=1, dilation=2)
        # self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.conv3 = nn.Conv1d(128, 256, kernel_size=16, stride=1, dilation=2)
        # self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.conv4 = nn.Conv1d(256, 512, kernel_size=32, stride=1, dilation=2)
        # self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.flatten = nn.Flatten()
        self.fc1 = None # This layer will be initialized in the forward method
        # self.drp1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        # self.drp2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, num_vars)
    
    def forward(self, x):
        x = self.conv1(x)
        # x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        # x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        # x = self.bn3(x)
        x = torch.relu(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        # x = self.bn4(x)
        x = torch.relu(x)
        x = self.pool4(x)
        
        x = self.flatten(x)
        
        # Initialize the first fully connected layer the first time forward is run
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)
            
        x = self.fc1(x)
        x = torch.relu(x)
        # x = self.drp1(x)
        
        x = self.fc2(x)
        x = torch.relu(x)
        # x = self.drp2(x)
        
        x = self.fc3(x)
        return x

    def initialize(self, x):
        """Initialize the model by passing an input tensor."""
        self.forward(x)
        print("\nModel initialized successfully.")

In [5]:
# Hybrid Deep Model with CNN and Transformer Encoder
class HybridDeepModel(nn.Module):
    def __init__(self, num_variables):
        super(HybridDeepModel, self).__init__()
        
        # CNN Layers
        self.conv1 = nn.Conv1d(1, 64, kernel_size=16, stride=1, dilation=1)
        self.pool1 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=16, stride=1, dilation=2)
        self.pool2 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.conv3 = nn.Conv1d(128, 256, kernel_size=16, stride=1, dilation=2)
        self.pool3 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        self.conv4 = nn.Conv1d(256, 512, kernel_size=32, stride=1, dilation=2)
        self.pool4 = nn.MaxPool1d(kernel_size=4, stride=4)

        # Normalization before Transformer
        self.norm = nn.LayerNorm(512)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=4, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(512, 128)  # Adjust input size based on the output of the last conv layer
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_variables)
    
    def forward(self, x):
        # CNN Feature Extraction
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = torch.relu(x)   
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = torch.relu(x)
        x = self.pool4(x) 
        
        # Prepare for Transformer
        # Transpose to match Encoder input shape: (batch_size, sequence_length, embed_dim)
        x = x.permute(0, 2, 1)

        x = self.norm(x)

        x = self.transformer(x)  # Output: (sequence_length, batch_size, embed_dim)
        x = x[:, -1, :]  # Take the last sequence element (batch_size, embed_dim)
        
        # Fully Connected Layers
        x = self.fc1(x)
        x = torch.relu(x)
        
        x = self.fc2(x)
        x = torch.relu(x)

        x = self.fc3(x)

        return x
    
    def initialize(self, x):
        """Initialize the model by passing an input tensor."""
        self.forward(x)
        print("Model initialized successfully.")

In [7]:
# Load your CSV
df = pd.read_csv('./real_gw_slice_analysis/real_gw_slice_summary.csv')
df

,model_name,slice_name,best_training_loss,best_train_epoch,best_validation_loss,best_val_epoch,boundary_test_loss
0,DeepModel,slice1,109.097900,1,147.708466,1,147.708461
1,DeepModel,slice2,148.686417,1,292.160706,1,292.160710
2,DeepModel,slice3,168.305664,1,374.932343,1,374.932324
3,DeepModel,slice4,143.230835,1,8701.465820,1,8701.465721
4,DeepModel,slice5,154.478973,1,672.786560,1,672.786526
5,DeepModel,slice1_reverse,171.326752,1,2253.885254,1,2253.885022
6,DeepModel,slice2_reverse,144.422348,1,154.501648,1,153.845628
7,DeepModel,slice3_reverse,149.135834,1,173.223511,1,173.223514
8,DeepModel,slice4_reverse,141.015533,1,142.678864,1,142.678852
9,DeepModel,slice5_reverse,151.645660,1,117.169907,1,117.169903


In [8]:
# Group by model and slice_type to compute mean MAE Loss and std.
df['slice_type'] = df['slice_name'].apply(lambda x: 'reverse' if 'rev' in x else 'normal')

stats = df.groupby(['model_name', 'slice_type'])['best_validation_loss'].agg(['mean', 'std']).reset_index()

print(stats)

        model_name slice_type         mean          std
0        DeepModel     normal  2037.810779  3730.031342
1        DeepModel    reverse   568.291837   942.494545
2  HybridDeepModel     normal   148.362421     9.963584
3  HybridDeepModel    reverse   147.443047    23.717795


In [9]:
# Group by model and slice_type to compute mean MAE Loss and std for boundary approach.
bound_stats = df.groupby(['model_name', 'slice_type'])['boundary_test_loss'].agg(['mean', 'std']).reset_index()

print(bound_stats)

        model_name slice_type         mean          std
0        DeepModel     normal  2037.810748  3730.031303
1        DeepModel    reverse   568.160584   942.566490
2  HybridDeepModel     normal   148.348948     9.947642
3  HybridDeepModel    reverse   147.388370    23.644483


In [11]:
# make predictions on real GW data for all slices and both models for supplementary material
slice_names = ['slice1', 'slice2', 'slice3', 'slice4', 'slice5',
               'slice1_reverse', 'slice2_reverse', 'slice3_reverse', 'slice4_reverse', 'slice5_reverse']

for model_name in ['DeepModel', 'HybridDeepModel']:
    for slice_name in slice_names:
        path = '/home/sakellariou/hero_disk/test/'  # <--- Change this to the path of the dataset
        csv_path = path + 'real_gw.csv'

        real_train_dataset = IterTensorFlowToPyTorchDataset(path + f'{slice_name}_train_dataset')
        # print('\nNumber of training samples in real dataset:', len(real_train_dataset))
        # print(f'\nSize of real training dataset: {real_train_dataset.size(in_gb=False):.3f} MB')

        real_test_dataset = IterTensorFlowToPyTorchDataset(path + f'{slice_name}_test_dataset')
        # print('\nNumber of test samples in real dataset:', len(real_test_dataset))
        # print(f'\nSize of real test dataset: {real_test_dataset.size(in_gb=False):.3f} MB')

        # Create DataLoader for batching
        # print('\nCreating DataLoaders...')
        batch = 1024
        real_train_loader = DataLoader(real_train_dataset, batch_size=batch, shuffle=False)
        real_test_loader = DataLoader(real_test_dataset, batch_size=batch, shuffle=False)
        
        # Initialize model, loss function, and optimizer
        num_vars = 3
        models = {
            'DeepModel': DeepModel,
            'HybridDeepModel': HybridDeepModel
        }

        model = models[model_name](num_vars).to(device)
        
        weights_path = f'./real_gw_slice_analysis/{slice_name}_{model.__class__.__name__}_results/{model.__class__.__name__}_best_model.pth' # <-- Change this to the path of the best model weights


        # Load the best model weights
        print('\nLoading best model weights...')
        try:
            model.load_state_dict(torch.load(f'{weights_path}'))
            print('\nModel weights loaded successfully.')
        except:
            print('\nError loading model weights. Initializing model...')
            with torch.no_grad():
                inputs, targets = next(iter(real_train_loader))
                inputs = inputs.to(device)
                model.initialize(inputs)
            model.load_state_dict(torch.load(f'{weights_path}'))
            print('\nModel weights loaded successfully.')
    
    
        model.eval()
        
        # import parameters from csv
        df = pd.read_csv(csv_path)
        df['keys'] = df['row_key'].astype(str).apply(lambda x: x.split('_', 1)[1])
        df.set_index('keys', inplace=True)
        
        # Per-value accumulators
        mass_1_index = 0  # Mass 1 is the first variable (index 0)
        mass_2_index = 1  # Mass 2 is the second variable (index 1)
        distance_index = 2  # Distance is the third variable (index 2)
        
        mae_sum = np.zeros(num_vars)
        bound_mae_sum = np.zeros(num_vars)
        count = 0

        with torch.no_grad():
            progress_bar = tqdm(real_test_loader, desc="Evaluating", dynamic_ncols=True)

            for inputs, targets in progress_bar:
                inputs, targets = inputs.to(device), targets.to(device)
                test_predictions = model(inputs)

                y_true = targets.cpu().numpy().astype(np.float64)
                y_pred = test_predictions.cpu().numpy().astype(np.float64)

                # Adjust predictions based on the parameters from the CSV file
                keys = [f'{m1:.2f}_{m2:.2f}_{d:.1f}' for m1, m2, d in zip(y_true[:, 0].round(2), y_true[:, 1].round(2), y_true[:, 2].round(1))]

                df_batch = df[df.index.isin(keys)].reindex(keys)
                
                mae_sum += np.sum(np.abs(y_true - y_pred), axis=0)
                count += y_true.shape[0]
                
                # Adjust predictions based on the bounds on the parameters from the CSV file
                #-------------------------------------------------------------------
                # print(df_batch)  # Debugging line to check the batch DataFrame
                m1_low = df_batch['mass_1'].values + df_batch['mass_1_low'].values
                m1_high = df_batch['mass_1'].values + df_batch['mass_1_high'].values
                m2_low = df_batch['mass_2'].values + df_batch['mass_2_low'].values
                m2_high = df_batch['mass_2'].values + df_batch['mass_2_high'].values
                distance_low = df_batch['distance'].values + df_batch['distance_low'].values
                distance_high = df_batch['distance'].values + df_batch['distance_high'].values
                
                mass1 = y_true[:, mass_1_index]
                mass2 = y_true[:, mass_2_index]
                distance = y_true[:, distance_index]

                pr_mass1 = y_pred[:, mass_1_index]
                pr_mass2 = y_pred[:, mass_2_index]
                pr_distance = y_pred[:, distance_index]

                in_m1 = (pr_mass1 >= m1_low) & (pr_mass1 <= m1_high)
                in_m2 = (pr_mass2 >= m2_low) & (pr_mass2 <= m2_high)
                in_distance = (pr_distance >= distance_low) & (pr_distance <= distance_high)
                
                adjusted_pred = y_pred.copy()
                # adjusted_pred[in_all] = y_true[in_all]  # Adjust predictions to match true values where conditions are met
                adjusted_pred[in_m1, mass_1_index] = y_true[in_m1, mass_1_index]
                adjusted_pred[in_m2, mass_2_index] = y_true[in_m2, mass_2_index]
                adjusted_pred[in_distance, distance_index] = y_true[in_distance, distance_index]
                
                bound_mae_sum += np.sum(np.abs(y_true - adjusted_pred), axis=0)
                #-------------------------------------------------------------------
                
                m1 = y_true[:, mass_1_index]
                m1_low_value = df_batch['mass_1_low'].values
                m1_high_value = df_batch['mass_1_high'].values
                m2 = y_true[:, mass_2_index]
                m2_low_value = df_batch['mass_2_low'].values
                m2_high_value = df_batch['mass_2_high'].values
                d = y_true[:, distance_index]
                d_low_value = df_batch['distance_low'].values
                d_high_value = df_batch['distance_high'].values
                pr_m1 = y_pred[:, mass_1_index]
                pr_m2 = y_pred[:, mass_2_index]
                pr_d = y_pred[:, distance_index]
                event_names = df_batch['event_name'].values
                
                print(f'{model_name} - {slice_name}:')
                for i in range (len(m1)):
                    print(f'\nEvent: {event_names[i]}')
                    print(f'Mass 1 true: {m1[i]} lower: {m1_low_value[i]} upper: {m1_high_value[i]} predicted: {pr_m1[i]}')
                    print(f'Mass 2 true: {m2[i]} lower: {m2_low_value[i]} upper: {m2_high_value[i]} predicted: {pr_m2[i]}')
                    print(f'Distance true: {d[i]} lower: {d_low_value[i]} upper: {d_high_value[i]} predicted: {pr_d[i]}')
                print()
            
            # Final metrics
            mae = mae_sum / count
            bound_mae = bound_mae_sum / count
            total_mae_loss = np.sum(mae) / num_vars
            total_bound_mae_loss = np.sum(bound_mae) / num_vars
            print(f'{model_name} - {slice_name} Test Loss: {total_mae_loss:.4f} Bound Test Loss: {total_bound_mae_loss:.4f}')
            print('-------------------------------------------------------------------')
    

                


Loading best model weights...

Error loading model weights. Initializing model...


2025-11-22 17:22:31.780835: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Model initialized successfully.

Model weights loaded successfully.


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.21it/s]


DeepModel - slice1:

Event: GW151226-v2
Mass 1 true: 13.699999809265137 lower: -3.2 upper: 8.8 predicted: -5.610050201416016
Mass 2 true: 7.699999809265137 lower: -2.5 upper: 2.2 predicted: -3.0104382038116455
Distance true: 450.0 lower: -190.0 upper: 180.0 predicted: 6.870304107666016

Event: GW150914-v3
Mass 1 true: 35.599998474121094 lower: -3.1 upper: 4.7 predicted: -2.057086229324341
Mass 2 true: 30.600000381469727 lower: -4.4 upper: 3.0 predicted: 1.5809446573257446
Distance true: 440.0 lower: -170.0 upper: 150.0 predicted: 10.659465789794922

Event: GW200202_154313-v1
Mass 1 true: 10.100000381469727 lower: -1.4 upper: 3.5 predicted: -2.2025625705718994
Mass 2 true: 7.300000190734863 lower: -1.7 upper: 1.1 predicted: -3.645237922668457
Distance true: 410.0 lower: -160.0 upper: 150.0 predicted: 2.849111318588257

Event: GW191216_213338-v1
Mass 1 true: 12.100000381469727 lower: -2.2 upper: 4.6 predicted: -24.22264289855957
Mass 2 true: 7.699999809265137 lower: -1.9 upper: 1.6 predi

...

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  7.67it/s]

HybridDeepModel - slice4_reverse:

Event: GW200202_154313-v1
Mass 1 true: 10.100000381469727 lower: -1.4 upper: 3.5 predicted: 35.11842346191406
Mass 2 true: 7.300000190734863 lower: -1.7 upper: 1.1 predicted: 5.146790981292725
Distance true: 410.0 lower: -160.0 upper: 150.0 predicted: -15.835030555725098

HybridDeepModel - slice4_reverse Test Loss: 151.0022 Bound Test Loss: 151.0022
-------------------------------------------------------------------



Loading best model weights...

Model weights loaded successfully.


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  7.16it/s]

HybridDeepModel - slice5_reverse:

Event: GW191216_213338-v1
Mass 1 true: 12.100000381469727 lower: -2.2 upper: 4.6 predicted: -23.808917999267578
Mass 2 true: 7.699999809265137 lower: -1.9 upper: 1.6 predicted: -22.888227462768555
Distance true: 340.0 lower: -130.0 upper: 120.0 predicted: -1.7219287157058716

HybridDeepModel - slice5_reverse Test Loss: 136.0730 Bound Test Loss: 136.0730
-------------------------------------------------------------------
